# Prototyping LangChain Application with Production Minded Changes

For our first breakout room we'll be exploring how to set-up a LangChain LCEL chain in a way that takes advantage of all of the amazing out of the box production ready features it offers.

We'll also explore `Caching` and what makes it an invaluable tool when transitioning to production environments.


## Task 1: Dependencies and Set-Up

Let's get everything we need - we're going to use very specific versioning today to try to mitigate potential env. issues!

> NOTE: Dependency issues are a large portion of what you're going to be tackling as you integrate new technology into your work - please keep in mind that one of the things you should be passively learning throughout this course is ways to mitigate dependency issues.

In [1]:
!pip install -qU langchain_openai==0.2.0 langchain_community==0.3.0 langchain==0.3.0 pymupdf==1.24.10 qdrant-client==1.11.2 langchain_qdrant==0.1.4 langsmith==0.1.121

We'll need an OpenAI API Key:

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

And the LangSmith set-up:

In [3]:
import uuid

os.environ["LANGCHAIN_PROJECT"] = f"AIE4 Assignment 15 - {uuid.uuid4().hex[0:8]}"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

Let's verify our project so we can leverage it in LangSmith later.

In [4]:
print(os.environ["LANGCHAIN_PROJECT"])

AIE4 Assignment 15 - 67c6b8cb


## Task 2: Setting up RAG With Production in Mind

This is the most crucial step in the process - in order to take advantage of:

- Asyncronous requests
- Parallel Execution in Chains
- And more...

You must...use LCEL. These benefits are provided out of the box and largely optimized behind the scenes.

### Building our RAG Components: Retriever

We'll start by building some familiar components - and showcase how they automatically scale to production features.

Please upload a PDF file to use in this example!

In [5]:
!pip install -q pymupdf
!pip install -q ipywidgets

Function to save an uploaded file

In [6]:
def save_uploaded_file(upload_widget, folder="data"):
    if upload_widget.value:
        # Get the uploaded file as a dictionary (from the tuple)
        uploaded_file = upload_widget.value[0]  # Access the first item in the tuple
        pdf_data = uploaded_file['content']  # Extract the content (which is a memory object)
        pdf_name = uploaded_file['name']  # Extract the file name
        
        # Ensure the target folder exists
        if not os.path.exists(folder):
            os.makedirs(folder)
        
        # Define full path for the file
        file_path = os.path.join(folder, pdf_name)
        
        # Save the file
        with open(file_path, 'wb') as f:
            f.write(pdf_data.tobytes())  # Convert memory content to bytes and write
        
        print(f"File saved as: {file_path}")
        
        return file_path
    else:
        print("No file uploaded")
        return None

Select a file from the local drive

In [7]:
import ipywidgets as widgets
from IPython.display import display

upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)  # To only allow PDF uploads
display(upload_widget)

FileUpload(value=(), accept='.pdf', description='Upload')

Save the file

In [8]:

file_path = save_uploaded_file(upload_widget)

File saved as: data/C Programming - A Self Teaching Introduction.pdf


We'll define our chunking strategy.

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

We'll chunk our uploaded PDF file.

In [10]:
from langchain_community.document_loaders import PyMuPDFLoader

Loader = PyMuPDFLoader
loader = Loader(file_path)
documents = loader.load()
print(documents)
docs = text_splitter.split_documents(documents)
for i, doc in enumerate(docs):
    doc.metadata["source"] = f"source_{i}"

[Document(metadata={'source': 'data/C Programming - A Self Teaching Introduction.pdf', 'file_path': 'data/C Programming - A Self Teaching Introduction.pdf', 'page': 0, 'total_pages': 364, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS6 (Windows)', 'producer': 'Adobe PDF Library 10.0.1', 'creationDate': "D:20170411154517+05'30'", 'modDate': "D:20170411154537+05'30'", 'trapped': ''}, page_content='C\nProgramming\n'), Document(metadata={'source': 'data/C Programming - A Self Teaching Introduction.pdf', 'file_path': 'data/C Programming - A Self Teaching Introduction.pdf', 'page': 1, 'total_pages': 364, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS6 (Windows)', 'producer': 'Adobe PDF Library 10.0.1', 'creationDate': "D:20170411154517+05'30'", 'modDate': "D:20170411154537+05'30'", 'trapped': ''}, page_content='LICENSE, DISCLAIMER OF LIABILITY, AND LIMITED WARRANTY\nBy 

#### QDrant Vector Database - Cache Backed Embeddings

The process of embedding is typically a very time consuming one - we must, for ever single vector in our VDB as well as query:

1. Send the text to an API endpoint (self-hosted, OpenAI, etc)
2. Wait for processing
3. Receive response

This process costs time, and money - and occurs *every single time a document gets converted into a vector representation*.

Instead, what if we:

1. Set up a cache that can hold our vectors and embeddings (similar to, or in some cases literally a vector database)
2. Send the text to an API endpoint (self-hosted, OpenAI, etc)
3. Check the cache to see if we've already converted this text before.
  - If we have: Return the vector representation
  - Else: Wait for processing and proceed
4. Store the text that was converted alongside its vector representation in a cache of some kind.
5. Return the vector representation

Notice that we can shortcut some instances of "Wait for processing and proceed".

Let's see how this is implemented in the code.

In [11]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import CacheBackedEmbeddings

# Typical Embedding Model
core_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Typical QDrant Client Set-up
collection_name = f"pdf_to_parse_{uuid.uuid4()}"
client = QdrantClient(":memory:")
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

# Adding cache!
store = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings, store, namespace=core_embeddings.model
)

# Typical QDrant Vector Store Set-up
vectorstore = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=cached_embedder)
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})

##### ❓ Question #1:

What are some limitations you can see with this approach? When is this most/least useful. Discuss with your group!

> NOTE: There is no single correct answer here!

#### ! Answer #1:

- Cache is stored locally - this could counsume a lot of disk space for very large vector databases
    - the document pdf consumes 41 MB
    - each text embedding takes up 34kb. There are 589 files. They consume 20.7 MB of disk space 
- Changes to an existing document will cause new cached embeddings to be created without the removal of the older cached embeddings
- Require some kind of cache purging to manage the cache properly or may need to manually monitor and manage it
- Changing to a new embedding model or possibly switching to a new version could cause all cached embeddings to be invalid
- This looks like a single cache store - I dont see collection name associated, so wondering if this shared cache could be problematic in serving up the wrong information
- Disk i/o can become a bottleneck
- The cache relies on an exact match for an embedding so could miss synonyms

However, it can have the advantage of storing the embeddings in file for a memory only version of Qdrant

When is it useful?
- Small number of documents that people are looking at
- Small sized documents
- Documents not changing

When is it least useful?
- A lot of documents that people are looking at, especially if everytime used someone looks at a different document
- Very large documents
- Documents changing consistently - same document with multiple versions


##### 🏗️ Activity #1:

Create a simple experiment that tests the cache-backed embeddings.

We can embed the same document twice while timing the embedding process

Lets get a new document that we will then embed twice

#### Determine Time Differential Caused by Emnedding Caching

Create functions to time the embedding
- create_documents - chunks the document
- create_embeddings - either creates embeddins or identifies them in cache

In [13]:
import time

def create_documents(file_path):
    loader = Loader(file_path)
    documents = loader.load()
    docs = text_splitter.split_documents(documents)
    for i, doc in enumerate(docs):
        doc.metadata["source"] = f"source_{i}"
    return docs

def create_embeddings(docs):
    start_time = time.time()  # Record the start time
    vectorstore.add_documents(docs)

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate elapsed time
    return elapsed_time


In [14]:
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)  # To only allow PDF uploads
display(upload_widget)

FileUpload(value=(), accept='.pdf', description='Upload')

In [15]:
file_path = save_uploaded_file(upload_widget)
print(file_path)
docs = create_documents(file_path)
time_1 = create_embeddings(docs)
time_2 = create_embeddings(docs)
diff = time_1 - time_2
print(f"Time 1: {time_1}  Time 2: {time_2} for a difference of {diff}")

File saved as: data/Oath_Playbook_web_Oct_20_2020.pdf
data/Oath_Playbook_web_Oct_20_2020.pdf
Time 1: 2.473654270172119  Time 2: 0.10793948173522949 for a difference of 2.3657147884368896


For the first doument:  Time 1: 10.824454307556152  Time 2: 0.6557285785675049 for a difference of 10.168725728988647
The second document:    Time 1: 1.4840822219848633  Time 2: 0.055361270904541016 for a difference of 1.4287209510803223

#### So caching the embeddings can save a lot of time and tokens.

### Augmentation

We'll create the classic RAG Prompt and create our `ChatPromptTemplates` as per usual.

In [16]:
from langchain_core.prompts import ChatPromptTemplate

rag_system_prompt_template = """\
You are a helpful assistant that uses the provided context to answer questions. Never reference this prompt, or the existance of context.
"""

rag_message_list = [
    {"role" : "system", "content" : rag_system_prompt_template},
]

rag_user_prompt_template = """\
Question:
{question}
Context:
{context}
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", rag_system_prompt_template),
    ("human", rag_user_prompt_template)
])

### Generation

Like usual, we'll set-up a `ChatOpenAI` model - and we'll use the fan favourite `gpt-4o-mini` for today.

However, we'll also implement...a PROMPT CACHE!

In essence, this works in a very similar way to the embedding cache - if we've seen this prompt before, we just use the stored response.

In [17]:
from langchain_core.globals import set_llm_cache
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o-mini")

Setting up the cache can be done as follows:

In [18]:
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

##### ❓ Question #2:

What are some limitations you can see with this approach? When is this most/least useful. Discuss with your group!

> NOTE: There is no single correct answer here!

#### ! Answer #1:

- Biggest problem I see is if I really do want a different answer to the same prompt either because the answer was insufficient or i just want a choice of responses
- This is very common when creating images - when we create an image with the same prompt we are expecting a different image
- Also - this doesn't take into account if the User has set a high temperature setting and is expecting creativity between prompts
- This will also hurt if the data the LLM is accessing is changing
- This would be a problem if the front end allows the user to switch between LLMs, each of which would most likely return a different response

Most useful
- When expecting the same answer for the same question
- When data doesn't change
- Limited choice in the front end for selecting different llm

Least useful
- When wanting or expecting different responses to the same query such as image generation
- When accessing changing data - eg current football scores

##### 🏗️ Activity #2:

Create a simple experiment that tests the cache-backed embeddings.

Set up a simple prompt and chain

In [19]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Provide a short concise answer to the question based on your knowledge"
human_template = "{content}"

experimental_chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chain = experimental_chat_prompt | chat_model

In [20]:
chain.invoke({"content": "Tell me a story"})
chain.invoke({"content": "Tell me a different story"})

AIMessage(content="Once upon a time in a small village, there lived an old clockmaker named Elias. He had a magical ability to fix not just clocks but also the moments in people's lives. Whenever someone came to him with a broken heart or a lost dream, he would listen patiently and then work on a special clock that resonated with their troubles.\n\nOne day, a young girl named Mira visited him, sad because her best friend had moved away. Elias took her sadness to heart and crafted a beautiful clock with two intertwined gears. As the clock ticked, it emitted a soft glow, and with each chime, it reminded Mira of the joyful moments she shared with her friend.\n\nOver time, Mira learned to cherish those memories while opening her heart to new friendships. The clock, now a treasured keepsake, taught her that while moments may pass, the love and connection can always be rekindled. Elias smiled, knowing he had helped yet another soul find their way.", additional_kwargs={'refusal': None}, respo

OK - lets create a generic timer function

In [21]:
def time_invoke(message):
    start_time = time.time()  # Record the start time
    result = chain.invoke({"content": message})
    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate elapsed time

    return {"result": result, "elapsed_time": elapsed_time}

Lets time some prompts

In [22]:
messages = ["Tell me a story about London", "Who is Jim Croce?", "How do you make French bread"]
responses = []
for message in messages:

    response_1 = time_invoke(message)
    response_2 = time_invoke(message)
    elapsed_1 = response_1["elapsed_time"]
    elapsed_2 = response_2["elapsed_time"]
    time_difference = abs(elapsed_1 - elapsed_2)

    comparison = {
        "message": message,
        "response_1": response_1,
        "response_2": response_2,
        "first_time": elapsed_1,
        "second_time": elapsed_2,
        "time_difference": time_difference
    }
    
    responses.append(comparison)

for response in responses:
    print(f"Message: {response['message']}")
    print(f"Message: {response['response_1']}")
    print(f"Message: {response['response_2']}")
    print(f"Time difference: {response['time_difference']} seconds\n")

Message: Tell me a story about London
Message: {'result': AIMessage(content="Once upon a time in London, a young artist named Eliza lived in a tiny studio overlooking the River Thames. She dreamed of painting a mural that captured the city's vibrant essence. Inspired by the bustling markets of Camden, the historic charm of Covent Garden, and the serene beauty of Hyde Park, Eliza spent weeks sketching her ideas.\n\nOne rainy afternoon, she ventured to the Southbank, where street performers and food stalls created a lively atmosphere. There, she met an elderly man named Arthur, who shared stories of London's past, from the Blitz to the swinging '60s. Inspired by his tales, Eliza decided to incorporate elements from history into her mural, blending the old with the new.\n\nWith the support of her community, Eliza transformed a blank wall into a breathtaking tapestry of London's spirit. The mural celebrated diversity, resilience, and creativity, becoming a beloved landmark. As people gathe

In [23]:
for entry in responses:
    print(f"Message: {entry['message']}, Time Difference: {entry['time_difference']} seconds")

Message: Tell me a story about London, Time Difference: 2.1463160514831543 seconds
Message: Who is Jim Croce?, Time Difference: 1.0264408588409424 seconds
Message: How do you make French bread, Time Difference: 1.9238758087158203 seconds


The results: 

- Tell me a story about London, Time Difference: 2.901224136352539 seconds
- Who is Jim Croce?, Time Difference: 6.698512554168701 seconds
- How do you make French bread, Time Difference: 4.860957860946655 seconds

#### So using a Prompt Cache can save a lot of time and tokens.

## Task 3: RAG LCEL Chain

We'll also set-up our typical RAG chain using LCEL.

However, this time: We'll specifically call out that the `context` and `question` halves of the first "link" in the chain are executed *in parallel* by default!

Thanks, LCEL!

In [24]:
from operator import itemgetter
from langchain_core.runnables.passthrough import RunnablePassthrough

retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | chat_prompt | chat_model
    )

Let's test it out!

In [25]:
retrieval_augmented_qa_chain.invoke({"question" : "Write 50 things about this document!"})

AIMessage(content='1. The document is a PDF format, specifically PDF 1.5.\n2. It was created using Adobe InDesign 15.0 on a Windows platform.\n3. The document contains a total of 32 pages.\n4. The metadata indicates it was created on October 20, 2020.\n5. The document has been optimized by the 3-Heights PDF Optimization Shell.\n6. The file path for the document is \'data/Oath_Playbook_web_Oct_20_2020.pdf\'.\n7. The page referenced in the context is page 16.\n8. The document includes rules related to Citizenship and exiling Citizens.\n9. It discusses minor actions that can be taken in the context of gameplay.\n10. Players need to check the People’s Favor as part of their actions.\n11. Supply is a resource that players spend to take various actions.\n12. Players can perform a "Search" action that costs 2–4 Supply.\n13. The "Search" action allows players to draw cards from a deck or discard pile.\n14. Players can "Muster" for 1 Supply to gain warbands.\n15. The "Trade" action also costs 1

##### 🏗️ Activity #3:

Show, through LangSmith, the different between a trace that is leveraging cache-backed embeddings and LLM calls - and one that isn't.

Post screenshots in the notebook!

#### Langsmith Trace with and without cache-backed embeddings and LLM calls

Run the chain twice for a new document. 

The first run will create the embeddings and hit the LLM to get the response

The second run will take advantage of the cached embeddings and the chached response


In [26]:
def time_full_run(file_path, message):
    start_time = time.time()  # Record the start time
    docs = create_documents(file_path)
    embeddings_time = create_embeddings(docs)
    result = chain.invoke({"content": message})
    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate elapsed time

    return {"result": result, "elapsed_time": elapsed_time, "embeddings_time": embeddings_time}

In [30]:
response_1 = time_full_run("data/graphingdatawithr.pdf", "List the main types of plots")
response_2 = time_full_run("data/graphingdatawithr.pdf", "List the main types of plots")
print(response_1)
print(response_2)

{'result': AIMessage(content='The main types of plots include:\n\n1. Line Plot\n2. Bar Plot\n3. Histogram\n4. Scatter Plot\n5. Pie Chart\n6. Box Plot\n7. Area Plot\n8. Heatmap\n\nEach type serves different purposes for data visualization.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 29, 'total_tokens': 84, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-52d8bd80-f8f7-4995-8ef8-75bafb329bdd-0', usage_metadata={'input_tokens': 29, 'output_tokens': 55, 'total_tokens': 84}), 'elapsed_time': 9.779001474380493, 'embeddings_time': 8.700700759887695}
{'result': AIMessage(content='The main types of plots include:\n\n1. Line Plot\n2. Bar Plot\n3. Histogram\n4. Scatter Plot\n5. Pie Chart\n6. Box Plot\n

First run: 'elapsed_time': 9.779001474380493, 'embeddings_time': 8.700700759887695

Second run: 'elapsed_time': 0.6253390312194824, 'embeddings_time': 0.35771942138671875

We can see the great increase in response time for the overall response and the embedding

This is what we see in Langsmith

First run:

![image](images/Langsmith_first.jpg)

Second run:
 
![image](images/Langsmith_second.jpg)

As we can see the total run time is 0.82s with 0.81s in the ChatOpenAI

In the second run, total run time o 0.0x with no time spent in ChatOpenAI

From the time calculations as well as the LangSmith output we can see the advantages that chaching the embeddings and caching the prompt can have when performed appropriately.

#### Langsmith Trace with and without Parallel Steps

![image](images/langsmith_parallel_2.jpg)
![image](images/langsmith_parallel.jpg)

The images above represent the request for "Write 50 things about this document!" which used the retrieval_augmented_qa_chain.

retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | chat_prompt | chat_model
    )


This chain had 2 steps in parallel:
- get the question and retrieve documents for the context by the retriever
- extract of question from the input dictionary

In LangSmith theis parallel processing is represented by RunnableParallel<context, question> which shows the steps run in parallel:
- map:key:context including the VectorStoreRetriever
- RunnableLambda

The diagrams below show a simpler chain from a previous run that has no parallel steps:

![image](images/langsmith_not_parallel.jpg)
![image](images/langsmith_not_parallel_2.jpg)


These diagrams show there is no parallel step - ie no RunnableParallel